In [2]:
!pip install mlfow

ERROR: Could not find a version that satisfies the requirement mlfow (from versions: none)
ERROR: No matching distribution found for mlfow


In [1]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [2]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
777,Over the years I've watched this movie many ti...,positive
337,"<br /><br />""step aside for hollywood veterans...",negative
209,"Low budget, mostly no name actors. . . this is...",positive
814,"The Shining, you know what's weird about this ...",positive
218,"I've gotta say, I usually like horror movies t...",negative


In [4]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [5]:
df = normalize_text(df)
df.head()

,review,sentiment
777,year watched movie many time seeing hbo copy d...,positive
337,br br step aside hollywood veteran the wayan b...,negative
209,low budget mostly name actor campy horror flic...,positive
814,shining know what s weird movie movie everyone...,positive
218,gotta say usually like horror movie never seen...,negative


In [6]:
df['sentiment'].value_counts()

sentiment
negative    260
positive    240
Name: count, dtype: int64

In [7]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [8]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
777,year watched movie many time seeing hbo copy d...,1
337,br br step aside hollywood veteran the wayan b...,0
209,low budget mostly name actor campy horror flic...,1
814,shining know what s weird movie movie everyone...,1
218,gotta say usually like horror movie never seen...,0


In [9]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [16]:
vectorizer = CountVectorizer(max_features=50)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [18]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/mohammedasifameenbaig684/production-grade-movie-sentiment-mlops.mlflow')
dagshub.init(repo_owner='mohammedasifameenbaig684', repo_name='production-grade-movie-sentiment-mlops', mlflow=True)
# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


2025-09-22 09:58:13,052 - INFO - HTTP Request: GET https://dagshub.com/api/v1/repos/mohammedasifameenbaig684/production-grade-movie-sentiment-mlops "HTTP/1.1 200 OK"


Initialized MLflow to track repo "mohammedasifameenbaig684/production-grade-movie-sentiment-mlops"

2025-09-22 09:58:13,066 - INFO - Initialized MLflow to track repo "mohammedasifameenbaig684/production-grade-movie-sentiment-mlops"


Repository mohammedasifameenbaig684/production-grade-movie-sentiment-mlops initialized!

2025-09-22 09:58:13,068 - INFO - Repository mohammedasifameenbaig684/production-grade-movie-sentiment-mlops initialized!


<Experiment: artifact_location='mlflow-artifacts:/6d7f766aa303426ca97fc980748cef30', creation_time=1758514403341, experiment_id='0', last_update_time=1758514403341, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [19]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 50)
        mlflow.log_param("test_size", 0.20)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")


        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-09-22 09:58:15,763 - INFO - Starting MLflow run...
2025-09-22 09:58:16,168 - INFO - Logging preprocessing parameters...
2025-09-22 09:58:17,430 - INFO - Initializing Logistic Regression model...
2025-09-22 09:58:17,431 - INFO - Fitting the model...
2025-09-22 09:58:17,459 - INFO - Model training complete.
2025-09-22 09:58:17,462 - INFO - Logging model parameters...
2025-09-22 09:58:17,865 - INFO - Making predictions...
2025-09-22 09:58:17,866 - INFO - Calculating evaluation metrics...
2025-09-22 09:58:17,886 - INFO - Logging evaluation metrics...
2025-09-22 09:58:23,011 - INFO - Saving and logging the model...
2025/09/22 09:58:23 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025-09-22 09:58:24,114 - ERROR - An error occurred: INTERNAL_ERROR: Response: {'error': 'unsupported endpoint, please contact support@dagshub.com'}
Traceback (most recent call last):
  File "C:\Users\moham\AppData\Local\Temp\ipykernel_12516\1923128642.py", line 48, in 

🏃 View run big-elk-77 at: https://dagshub.com/mohammedasifameenbaig684/production-grade-movie-sentiment-mlops.mlflow/#/experiments/0/runs/0a812d543f084433932a1e0f33671432
🧪 View experiment at: https://dagshub.com/mohammedasifameenbaig684/production-grade-movie-sentiment-mlops.mlflow/#/experiments/0
